In [1]:
import pandas as pd, numpy as np
import re, glob

### read in csv files with manifestos

In [2]:
eu = [
    "Austria", "Belgium", "Bulgaria","Croatia","Cyprus","Czech Republic","Denmark","Estonia","Finland","France","Germany",
    "Greece","Hungary","Ireland","Italy","Latvia","Lithuania","Luxembourg","Malta",
    "Netherlands","Poland","Portugal","Romania","Slovakia","Slovenia","Spain","Sweden",'United Kingdom']

In [3]:
master = pd.DataFrame()
codes = []
for x in glob.glob('per_country_full/*.csv'):
    country = x.split('/')[1].split('_')[0]
    if country in eu:
        print(country)
        df = pd.read_csv(x)
        print(df.shape[0])
        df = df.drop(columns=['Unnamed: 0'])
        master = pd.concat([master,df])
    

Austria
45387
Sweden
17387
Estonia
16813
Slovenia
40029
Croatia
31161
Italy
22702
Cyprus
12655
Portugal
48184
Netherlands
132586
Romania
11339
Greece
34763
Germany
88750
Ireland
30425
Belgium
155330
Czech Republic
26224
Finland
22643
France
9421
Denmark
19049
Hungary
47666
Poland
29081
Bulgaria
12369
Slovakia
30484
Spain
149575
Latvia
2031
Luxembourg
30845
Lithuania
44980
United Kingdom
18158


In [4]:
master.shape

(1130037, 15)

In [5]:
master['year']=master['manifesto_id'].apply(lambda x: pd.to_datetime(x.split("_")[1][:4]).year)
master['party_id']=master['manifesto_id'].apply(lambda x: x.split("_")[0])

In [6]:
# DROP ALL BEFORE 1990
master = master[master['year']>=1990]
master.shape

(1115438, 17)

In [7]:
master.reset_index(inplace=True)

In [8]:
# manifestos per language
master['language'].value_counts()

language
dutch         201807
german        156427
spanish       120404
french         90682
english        49287
portuguese     48149
hungarian      47666
greek          47418
lithuanian     44980
slovenian      40029
croatian       31161
slovak         30261
polish         28550
czech          26224
catalan        23182
finnish        22303
italian        21836
danish         18934
swedish        17624
estonian       16813
bulgarian      12365
romanian       11339
galician        5966
latvian         2031
Name: count, dtype: int64

In [9]:
# manifestos per country
master.groupby(['country'])['manifesto_id'].nunique()

country
Austria            50
Belgium            79
Bulgaria           25
Croatia            54
Cyprus             20
Czech Republic     31
Denmark            79
Estonia            24
Finland            62
France             38
Germany            50
Greece             49
Hungary            28
Ireland            41
Italy              69
Latvia             30
Lithuania          42
Luxembourg         17
Netherlands        96
Poland             43
Portugal           41
Romania            29
Slovakia           46
Slovenia           41
Spain             104
Sweden             60
United Kingdom     22
Name: manifesto_id, dtype: int64

In [10]:
# DROP ALL THAT HAVEN'T BEEN ANNOTATED AT QS LEVEL, WE WILL COME BACK TO THESE IN LATER SCRIPTS
mvc = master['manifesto_id'].value_counts()
master = master[~master['manifesto_id'].isin(mvc[mvc==1].index)]
master.reset_index(inplace=True)

In [11]:
master.shape

(1115148, 19)

In [ ]:
# create QS id 
newm = pd.DataFrame()
for i,r in enumerate(master['manifesto_id'].unique()):
    s = master[master['manifesto_id']==r]
    qs = [r+'_'+str(0)*(4-len(str(x)))+str(x) for x in range(s.shape[0])]
    s['qs_new']=qs
    newm = pd.concat([newm,s])
    # for q in qs:
    #     qs_ids.append(q)

In [14]:
newm.reset_index(inplace=True,drop=True)

In [15]:
newm.shape, master.shape

((1115148, 20), (1115148, 19))

In [ ]:
# save 
newm.to_csv('all_eu_raw_mpd.csv',header=True,index=False)

## fix slashes -- double slashes demark quasi-sentences in the annotatied pdfs. some have not been parsed correctly so we fix that here

In [ ]:
# how big is the issue?
sl_all = pd.Series([1 if '//' in x.strip('//') and 'HYPERLINK' not in x else 0 for x in newm['original_text']])
sl_all.sum()

In [ ]:
d['slash']=sl_all
pd.DataFrame([d[d['slash']==1]['qs_new'],d[d['slash']==1]['original_text']]).T.to_csv('bads.csv')

In [ ]:
# manaual evaluation of qs containing slashes that are actually multiple quasi-sentences, add 'fix_slash' column denoting these to be fixed

In [ ]:
tofix = pd.read_csv('bads_w_fix2.csv')

In [ ]:
fslsh = tofix[tofix['fix_slash']==1]
fslsh.shape

In [ ]:
# initiate new df to house split entries
new_entries = pd.DataFrame()
# iterate over qs that need to be fixed
for qs,text in zip(fslsh['qs_new'],fslsh['original_text']):
    # split text
    splits = text.split('//')
    #print(splits)
    
    # enumerate over splits
    for n,split in enumerate(splits):
        # check for empties 
        if len(split) < 1: 
            continue
            
        # extrct original from database, create new entries based on the original meta data but with new text and new qs 
        else:
            new = d[d['qs_new']==qs]
            new['original_text']=split
            new['qs_new']=new['qs_new']+f".{n}"

            # compile into new entries dataframe
            new_entries = pd.concat([new_entries,new])

orig_ind = new_entries.index
new_entries.reset_index(inplace=True,drop=True)

In [ ]:
new_entries.shape

In [ ]:
new_entries.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
orig_ind = orig_ind.unique()
orig_ind

In [ ]:
# concat new entries with master
d=pd.concat([newm,new_entries])
d.shape

In [ ]:
# drop original (bad) qs
d.drop(orig_ind,inplace=True)
d.shape

In [ ]:
# reset index of d
d.reset_index(inplace=True,drop=True)

#### remove slashes from rest of the ok ones

In [ ]:
# remove slashes
slsh = tofix.query('remove_slash == 1')
for i in slsh['qs_new']:
    t = d[d['qs_new']==i]['original_text'].tolist()[0]
    t = t.replace('//','')
    d.loc[d['qs_new']==i,'original_text']=t

In [ ]:
d.to_csv('cleaned_data.csv',index=False)

## keyword detection

In [16]:
# custom utils made to hold keyword dictionaries and extract function 
import kw_utils, importlib
from kw_utils import get_kw, extract_keywords

In [ ]:
gw_dict = get_kw('gw_dict')
cc_dict = get_kw('climate_change_dict')
c_dict = get_kw('climate_dict')
cer_dict = get_kw('clim_em_ren')
se_dict = get_kw('sust_env')

In [ ]:
# identify all matches se and clim em ren matches 
all_matches = pd.DataFrame()

for lang in d['language'].unique():
    print(lang)
    s = d[(d['language']==lang)]
    #print(s.shape)
     
    kws = gw_dict[lang] 
    matches = extract_keywords(s['original_text'],kws,list=True)
    s['kw_gw']=matches

    kws = cc_dict[lang] 
    matches = extract_keywords(s['original_text'],kws,list=True)
    s['kw_cc']=matches
    
    kws = c_dict[lang] 
    matches = extract_keywords(s['original_text'],kws,list=True)
    s['kw_c']=matches
    
    kws = cer_dict[lang]
    matches = extract_keywords(s['original_text'],kws,list=True)
    s['kw_cer']=matches
    
    kws = se_dict[lang]
    matches = extract_keywords(s['original_text'],kws,list=True)
    s['kw_se']=matches
    
    s=s[['qs_new','date','kw_gw','kw_cc','kw_c','kw_se','kw_cer','language','country']]
    all_matches=pd.concat([all_matches,s])

In [ ]:
d = pd.merge(d,all_matches[['kw_cc','kw_c','kw_gw','kw_cer','kw_se','qs_new']],on='qs_new',how='inner')

In [ ]:
d.to_csv('data_cleaned_kw.csv',index=False)